<a href="https://colab.research.google.com/github/drfperez/YoungPhotonicsCongress/blob/main/PhotonTunnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Technical Commentary: Wave Dynamics and Photonic Analogies
1. Wave Packet Propagation and Optical Analogies
The code simulates a Gaussian wave packet interacting with a potential barrier. In photonics, this is mathematically analogous to an electromagnetic pulse traveling through a medium with a varying refractive index. The "Potential Barrier" (V) in the code acts exactly like a region of different refractive index or a photonic crystal gap.
2. Tunneling and Evanescent Waves
When the "Barrier Height" (bh\_slider) is high, the simulation shows a portion of the wave reflecting and a portion "tunneling" through.
 * Photonics Link: This is the quantum-optical equivalent of Frustrated Total Internal Reflection (FTIR) or Evanescent Wave Coupling. Even if the barrier is "forbidden," the electromagnetic field (represented by psi) can penetrate and reappear on the other side if the barrier is thin enough (the "Barrier Width").
 * Guided Wave Optics: This principle is used in directional couplers in integrated photonics to transfer light between two adjacent fiber cores or waveguides.
3. The Mathematics of Dispersion
The core of the step() function calculates the Laplacian (\nabla^2 \psi). In photonics, this governs diffraction and dispersion.
 * Pulse Broadening: As the wave packet moves, you will notice it spreads out. This is a simulation of Group Velocity Dispersion (GVD), a critical factor in optical fiber communications where pulses widen and overlap, limiting data rates.
4. Transmission (T) and Reflection (R) Coefficients
The draw() function calculates T and R by summing the probability density on either side of the barrier.
 * Thin Film Optics: This is the same principle used to design anti-reflective coatings or dielectric mirrors. By adjusting the width and height (refractive index contrast), you are essentially tuning the optical impedance matching of the system.
Summary for Photonics
This code is a powerful visualization of how wave-particle duality and wave interference govern the behavior of photons in nanophotonic structures. It allows you to experiment with how the geometry of a device (Width) and its material properties (Height/Index) determine whether light is reflected or transmitted.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import threading
import time

# ===== Parameters =====
N=600
dt=0.1
sigma=15
x0=50
k0=0.35
barrierX=300

running=False
thread=None

# ===== Widgets =====
bw_slider=widgets.IntSlider(value=40,min=10,max=150,description='Barrier Width')
bh_slider=widgets.IntSlider(value=5,min=1,max=10,description='Barrier Height')

start_btn=widgets.Button(description='▶ Start')
pause_btn=widgets.Button(description='⏸ Pause')
reset_btn=widgets.Button(description='⏹ Reset')

# ===== State =====
def initState():
    global psiRe,psiIm,V,Tdata,Rdata
    psiRe=np.zeros(N)
    psiIm=np.zeros(N)
    V=np.zeros(N)

    for i in range(N):
        env=np.exp(-((i-x0)**2)/(2*sigma**2))
        psiRe[i]=env*np.cos(k0*i)
        psiIm[i]=env*np.sin(k0*i)

    updateBarrier()
    Tdata=[]
    Rdata=[]

def updateBarrier(change=None):
    V[:]=0
    V[barrierX:barrierX+bw_slider.value]=bh_slider.value

bw_slider.observe(updateBarrier,'value')
bh_slider.observe(updateBarrier,'value')

def absorb(i):
    d=min(i,N-i)
    return d/40 if d<40 else 1

def step():
    r=psiRe.copy()
    im=psiIm.copy()

    for i in range(1,N-1):
        lapR=r[i-1]-2*r[i]+r[i+1]
        lapI=im[i-1]-2*im[i]+im[i+1]

        psiRe[i]+=dt*(-0.5*lapI+V[i]*im[i])
        psiIm[i]-=dt*(-0.5*lapR+V[i]*r[i])

        a=absorb(i)
        psiRe[i]*=a
        psiIm[i]*=a

def draw():
    prob=psiRe**2+psiIm**2

    T=np.sum(prob[barrierX+bw_slider.value:])
    R=np.sum(prob[:barrierX])

    Tdata.append(T)
    Rdata.append(R)

    if len(Tdata)>N:
        Tdata.pop(0)
        Rdata.pop(0)

    fig,(ax1,ax2)=plt.subplots(2,1,figsize=(10,4))

    ax1.plot(prob*10)
    barrier=np.zeros(N)
    barrier[barrierX:barrierX+bw_slider.value]=bh_slider.value/10
    ax1.plot(barrier)

    ax1.set_xlim(0,N)
    ax1.set_ylim(0,1)

    ax2.plot(Tdata)
    ax2.plot(Rdata)
    ax2.set_ylim(0,1)

    plt.show()

# ===== Thread Loop =====
def sim_loop():
    global running
    while running:
        step()
        clear_output(wait=True)
        display(ui)
        draw()
        time.sleep(0.03)

# ===== Controls =====
def start(b):
    global running,thread
    if not running:
        running=True
        thread=threading.Thread(target=sim_loop)
        thread.start()

def pause(b):
    global running
    running=False

def reset(b):
    global running
    running=False
    initState()
    clear_output(wait=True)
    display(ui)
    draw()

start_btn.on_click(start)
pause_btn.on_click(pause)
reset_btn.on_click(reset)

# ===== UI =====
ui=widgets.VBox([
    bw_slider,
    bh_slider,
    widgets.HBox([start_btn,pause_btn,reset_btn])
])

# ===== Init =====
initState()
display(ui)
draw()